In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
 import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}\n".format(device))

Device: cpu



In [7]:
import pandas as pd
import pickle
import time
from torch.distributions import Categorical
from torch.distributions.uniform import Uniform
import torch.nn.functional as F
import numpy as np 

from src.synthetic_helpers import *
from src.SWkernels import k_sw_rf
from src.MMDkernels import k_MMD
from src.training import train_multiple_kernels
from src.classifiers import KRR

In [9]:
# GMM setup
nl = [50,250]#,100,250,500,1000] # samples/GMM
T = 100 # number of datasets
K = 2 # maximum number of components
d = 2 # dimension
num_exp = len(nl)
epoch = 5
T_val = 50
T_test = 100
dic_res = {"n":nl,"T":T,"K":K,"d":d,"SW2":[], "SW1":[],"MMD":[]}

In [10]:
means, covs, y = sample_moments(T, d, K_max=K)
means_val, covs_val, y_val = sample_moments(T_val, d, K_max=K)
means_test, covs_test, y_test = sample_moments(T_test, d, K_max=K)
for idx_exp in range(num_exp):
    n = nl[idx_exp]
    print("Starting with T= ", T, " n=" , n, " K= ", K, " d= ", d)
    X = sample_from_moments(means, covs, n, K, epoch)
    X_val = sample_from_moments(means_val, covs_val, n, K, epoch)
    X_test = sample_from_moments(means_test, covs_test, n, K, epoch)
    
    if "SW2" in dic_res:
        print("Starting SW2 for {} epochs".format(epoch))
        rmse = sw_training(X, y, X_val, y_val, X_test, y_test, 2, epoch) # SW2
        dic_res["SW2"].append(rmse)

    if "SW1" in dic_res:
        print("Starting SW1 for {} epochs".format(epoch))
        rmse = sw_training(X, y, X_val, y_val, X_test, y_test, 1, epoch) # SW1
        dic_res["SW1"].append(rmse)
    if "MMD" in dic_res:
        print("Starting MMD for {} epochs".format(epoch))
        rmse = mmd_training(X, y, X_val, y_val, X_test, y_test, epoch) # MMD
        dic_res["MMD"].append(rmse)
    pickle.dump(dic_res, open("results/gmm_res.p", "wb" ))

Starting with T=  100  n= 50  K=  2  d=  2
Starting SW2 for 5 epochs
SW epoch 1/5
Starting training...

time elapsed computing both Gram matrices 0.06s (shape: torch.Size([1, 100, 100]))

SW epoch 2/5
Starting training...

time elapsed computing both Gram matrices 0.05s (shape: torch.Size([1, 100, 100]))

SW epoch 3/5
Starting training...

time elapsed computing both Gram matrices 0.05s (shape: torch.Size([1, 100, 100]))

SW epoch 4/5
Starting training...

time elapsed computing both Gram matrices 0.05s (shape: torch.Size([1, 100, 100]))

SW epoch 5/5
Starting training...

time elapsed computing both Gram matrices 0.05s (shape: torch.Size([1, 100, 100]))

Starting SW1 for 5 epochs
SW epoch 1/5
Starting training...

time elapsed computing both Gram matrices 0.05s (shape: torch.Size([1, 100, 100]))

SW epoch 2/5
Starting training...

time elapsed computing both Gram matrices 0.05s (shape: torch.Size([1, 100, 100]))

SW epoch 3/5
Starting training...

time elapsed computing both Gram matr

- add verbose option
- add visualition of the table with pandas


In [8]:
dic_res

{'n': [5, 10, 50],
 'T': 10,
 'K': 2,
 'd': 2,
 'SW2': [[tensor(0.6325),
   tensor(0.5477),
   tensor(0.4472),
   tensor(0.7746),
   tensor(0.6325)],
  [tensor(0.7071),
   tensor(0.7071),
   tensor(0.5477),
   tensor(0.7071),
   tensor(0.5477)],
  [tensor(0.6325),
   tensor(0.6325),
   tensor(0.4472),
   tensor(0.6325),
   tensor(0.6325)]],
 'SW1': [[tensor(0.6325),
   tensor(0.4472),
   tensor(0.6325),
   tensor(0.7746),
   tensor(0.6325)],
  [tensor(0.5477),
   tensor(0.5477),
   tensor(0.4472),
   tensor(0.5477),
   tensor(0.6325)],
  [tensor(0.5477),
   tensor(0.6325),
   tensor(0.4472),
   tensor(0.5477),
   tensor(0.6325)]],
 'MMD': [[tensor(0.6325),
   tensor(0.5477),
   tensor(0.5477),
   tensor(0.6325),
   tensor(0.8367)],
  [tensor(0.6325),
   tensor(0.5477),
   tensor(0.6325),
   tensor(0.7071),
   tensor(0.7071)],
  [tensor(0.7746),
   tensor(0.7071),
   tensor(0.7746),
   tensor(0.7071),
   tensor(0.6325)]]}